In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import re
from scipy.optimize import curve_fit
import operator
from __future__ import division
import itertools
import pyfits

#critcurve_dir = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver21/reconstruction/critcurve_map_ver21_full.fits'
m_zp = 24.6949
gain = 2.5
t = 15785.05 #integration time (second)

#objID = [['6a','6b','6c']]
objID = [\
         ['1a','1b','1c'],\
         ['2a','2b','2c'],\
         ['3a','3b'],\
         ['4a','4b','4c'],\
         ['5a','5b'],\
         ['6a','6b','6c'],\
         ['7a','7b','7c'],\
         ['8a','8b'],\
         ['9a','9b','9c']\
        ] #the object ID of the lensed images
#adu = [[0.591616,0.434753,0.0723096]]
adu = [\
       [29.3226,30.7356,13.8437],\
       [6.82799,7.52682,2.98697],\
       [0.85418,1.68711],\
       [1.156,8.80144,2.58869],\
       [0.3249,0.162984],\
       [0.591616,0.49708,0.15351],\
       [4.5072,2.34779,2.28456944444],\
       [8.96649,17.6689],\
       [2.54068,1.26548,0.611643]\
      ] #total adu included inside the aperture of the object
    #[0.591616,0.434753,0.0723096],\ self-measured system 6

#sigma = [[0.00505569,0.00524056,0.00493436]]
sigma = [\
          [0.00488142,0.00494071,0.00496406],\
          [0.00503246,0.0050529,0.00509898],\
          [0.00502566,0.00502098],\
          [0.00481722,0.0048186,0.00480533],\
          [0.00485934,0.00478751],\
          [0.00505569,0.00524056,0.00493436],\
          [0.00487724,0.0048768,0.00489597],\
          [0.00489478,0.00484996],\
          [0.00479271,0.00484121,0.00480649]\
         ] #the average value of sky variance in the wht image

#Area = [[70,43,12]]
Area = [\
        [1774,1607,947],\
        [529,514,230],\
        [122,227],\
        [159,520,148],
        [57,35],\
        [70,43,12],\
        [272,250,221],\
        [676,791],\
        [203,147,109]\
       ] #the average value of sky variance in the wht image

#mu = [[5.54626,2.23642,1.35421]]
mu = [\
      [6.14,3.99,1.69],\
      [13.24,12.91,5.3],
      [25.57,43.84],\
      [0.86,8.49,3.18],\
      [3.57,2.91],\
      [10.6, 7.7, 2.6],\
      [9.67,6.61,2.45],\
      [47.2,90.1],\
      [3.6,3.22,4.54]\
     ] #the magnification value of different images real Universe (ver22, new)

#mu = [\
#      [2.45705,2.03967,1.25537],\
#      [3.6867,3.67825,2.23782],
#      [5.23505,5.99836],\
#      [0.88768,2.95514,1.80071],\
#      [1.90955,1.69824],\
#      [3.25238,2.69641,1.58626],\
#      [3.11517,2.63468,1.46607],\
#      [2.50365,12.6913],\
#      [1.94061,1.79737,2.09046]\
#     ] #the magnification value of different images real Universe (ver22, old)




#================================================================== 
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Read the Critical Curve File
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
#hdulist = pyfits.open(critcurve_dir)
#critcurve = (hdulist[0].data)

#mu = [\
#      [critcurve[1422,899],critcurve[546,1159],critcurve[1006,888]],\
#      [critcurve[1112,240],critcurve[1030,239],critcurve[513,373]],\
#      [critcurve[735,745],critcurve[693,779]],\
#      [critcurve[1066,1610],critcurve[863,1596],critcurve[1498,1406]],\
#      [critcurve[1186,1744],critcurve[707,1735]],\
#      [critcurve[727,597],critcurve[529,802],critcurve[1686,620]],\
#      [critcurve[1407,974],critcurve[535,1222],critcurve[1098,1147]],\
#      [critcurve[1344,1153],critcurve[1293,1252]],\
#      [critcurve[1107,1529],critcurve[535,1437],critcurve[1683,1118]]\
#     ] #ds9 pixel value - 1 for both x and y axes
#print(mu)

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Functions for basic mathematical operations
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

def PLUS(L1,L2):
    return [map(operator.add,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def plus(L1,n):
    return [[x + n for x in obj] for obj in L1]

def MINUS(L1,L2):
    return [map(operator.sub,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def minus(L1,n):
    return [[x - n for x in obj] for obj in L1]

def MULTIPLY(L1,L2):
    return [map(operator.mul,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def multiply(L1,n):
    return [[x * n for x in obj] for obj in L1]

def DIVIDE(L1,L2):
    return [map(operator.truediv,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def divide(L1,n):
    return [[x / n for x in obj] for obj in L1]

def square(L1):
    return [[x ** 2 for x in obj] for obj in L1]

def squareroot(L1):
    return [[x ** 0.5 for x in obj] for obj in L1]

def openup(L): # accept a list with tuple, return with a list without tuple
    temp = []
    for sys in L:
        temp = temp + list(sys)
    return np.asarray(temp)

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Scientific Functions
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

def magnitude(flux):  # return the magnitude correspond to the flux, input number only
    M = -2.5*(np.log10(flux)) + m_zp
    return M

def MAGNITUDE(flux):  # operate on whole list: [[],[],[]]
    return [[magnitude(x) for x in obj] for obj in flux]

def dM(flux, df):  # return the magnitude error based on flux and fluxErr, input number only
    err = 1.0851/flux * df
    return err

def DM(flux, df):  #operate on the whole list: [[],[],[]]
    return [map(dM,flux[flux.index(obj)],df[flux.index(obj)]) for obj in flux]



#================================================================== 

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Obtain the total error (ADUs^-1) from the wht image and object flux
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

OBJ1 = divide(adu, gain * t)
OBJ2 = MULTIPLY(square(Area),square(sigma))

NOISE = squareroot(PLUS(OBJ1,OBJ2))
#print(NOISE)
#================================================================== 


# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Get an array of object's magnitude and magnitude error
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
lum = multiply(adu, gain)  #change the total image flux to e-s^-1 unit
lumErr = multiply(NOISE, gain)  #change the total error to e-s^-1 unit

mag = MAGNITUDE(lum)  #change the luminosity list into magnitude
magErr = DM(lum,lumErr)  #change the lumErr list to magnitude error

print(' The magnitudes of sources are: ' + str(mag))
print(' The magnitudes error of sources are: ' + str(magErr))

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Functions to arrange the elements in appropriate form for plotting graph 1
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

def duplicate(data):  #rearrange the data, applicable to observeded magnitude, observed magnitude error
    #[['1a', '1b', '1c'], ['2a', '2b'] -> [1a, 1a, 1b, 1b, 1c, 1c, 2a, 2b]
    v = []
    for system in data:
        for value in system:
            for i in range(len(system)-1):
                v.append(value)
    return np.asarray(v)

def perm(data):  #rearrange the data, applicable to perdicted magnitude, predicted magnitude error
    #[['1a', '1b', '1c'], ['2a', '2b'] -> [1b, 1c, 1a, 1c, 1a, 1b, 2b, 2a]
    result = []
    for sys in data:
        temp = [np.concatenate((sys[:i],sys[i+1:])) for i in range(len(sys))]
        temp = np.array(temp).flatten()
        result = np.concatenate((result, temp))
    return result
    
def predicted(flux0, mu):  # predicted magnitude (order in perm(data))
    array_flux0 = perm(flux0)
    array_M0 = magnitude(array_flux0)
    array_mu0 = perm(mu)
    array_muI = duplicate(mu)
    predMag = array_M0 + 2.5*(np.log10(array_mu0/array_muI))
    return predMag

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Construct the x and y axis for the plot 1
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
x = duplicate(mag)
y = map(operator.sub,predicted(lum,mu),duplicate(mag))
yErr = map(operator.add,perm(magErr),duplicate(magErr))

#plot the graph
plt.errorbar(x, y, yerr = yErr, fmt='o')
#plt.axis([17, 29, 17, 29])
plt.xlabel('Observed Magnitude')
plt.ylabel('Predicted Magnitude - Observed Magnitude')
#plt.plot(range(100),range(100))
#plt.plot(fit_x, fit_y, 'r')
plt.show()


#================================================================== 
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Functions to align data in plot 2
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# Consider we have a data: L = [['1a','1b','1c'],['2a','2b','2c']],
# We make combinations of all elements inside the inner bracket, and store the elemnt separately into two arrays
# arrayI(L) = ['1b','1c','1c','2b','2c','2c']
# arrayO(L) = ['1a','1a','1b','2a','2a','2b']
def arrayI(L):
    I = []
    for sys in L:
        for C in itertools.combinations(sys,2):
            (itemO,itemI) = C
            I.append(itemI)
    return I

def arrayO(L):
    O = []
    for sys in L:
        for C in itertools.combinations(sys,2):
            (itemO,itemI) = C
            O.append(itemO)
    return O

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Construct the x and y axis for the plot 2
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
M_O = arrayO(mag)
M_I = arrayI(mag)
mu_O = arrayO(mu)
mu_I = arrayI(mu)
MErr_O = arrayO(magErr)
MErr_I = arrayI(magErr)

x_int = map(operator.truediv,mu_I,mu_O)
x = [-2.5 * np.log10(i) for i in x_int]
y = map(operator.sub, M_I, M_O)
yErr = map(operator.add,MErr_O,MErr_I)

plt.errorbar(x, y, yerr = yErr, fmt='o')

#calculate and plot best fit line

def fit_func(x, a, b):
    return a*x + b

params = curve_fit(fit_func, x, y)

[a, b] = params[0]

fit_x = np.arange(-7, 7)
fit_y = a*fit_x + b
plt.plot(fit_x, fit_y, 'r')
print(a)
print(b)

#plot a reference 1 to 1 line
x_ref = np.arange(-7,7)
y_ref = np.arange(-7,7)
plt.plot(x_ref, y_ref, 'green')
plt.xlabel('-2.5log(mu_p/mu_0)')
plt.ylabel('Predicted Magnitude - Observed Magnitude')
plt.axis([-3, 2, -4, 4])
save.fig('/Users/chanmingyan/Desktop/figure1.jpeg')
plt.show()


# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Construct the x and y axis for the plot 3 (Daniel's plot)
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
x = duplicate(mag)
xErr = duplicate(magErr)
y = predicted(lum,mu)
yErr = perm(magErr)

plt.errorbar(x, y, xerr = xErr, yerr = yErr, fmt='o')

#calculate and plot best fit line

def fit_func(x, a, b):
    return a*x + b

params = curve_fit(fit_func, x, y)

[a, b] = params[0]

fit_x = np.arange(0, 100)
fit_y = a*fit_x + b
plt.plot(fit_x, fit_y, 'r')

#plot a reference 1 to 1 line
x_ref = np.arange(0,100)
y_ref = np.arange(0,100)
plt.plot(x_ref, y_ref, 'green')

plt.xlabel('Observed Magnitude')
plt.ylabel('Predicted Magnitude')
plt.axis([19, 28, 19, 28])
plt.show()


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import math
import re
from scipy.optimize import curve_fit
import operator
from __future__ import division

adu = [\
       [40.0353,29.4606,19.8437],\
       [7.70359,9.625452,5.1469589],\
       [0.893754165102,1.87928501961],\
       [1,12.0289544153,3.48911362055],\
       [0.654471534472,0.464703521739],\
       [1.62446953174,1.71061766302,0.395701680637],\
       [10.5085617047,5.83603933067,2.48269709371],\
       [12.0657174981,11.9871428602],\
       [4.77205282147,0.872451539683,0.875281728573]\
      ] #total adu included inside the aperture of the object

test1 = [[1,2],[5,55,7],[1,4,0.1]]
test2 = [[8,1],[9,82,109],[0.1,1,4]]


def PLUS(L1,L2):
    return [map(operator.add,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def plus(L1,n):
    return [[x + n for x in obj] for obj in L1]

def MINUS(L1,L2):
    return [map(operator.sub,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def minus(L1,n):
    return [[x - n for x in obj] for obj in L1]

def MULTIPLY(L1,L2):
    return [map(operator.mul,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def multiply(L1,n):
    return [[x * n for x in obj] for obj in L1]

def DIVIDE(L1,L2):
    return [map(operator.truediv,L1[L1.index(obj)],L2[L1.index(obj)]) for obj in L1]

def divide(L1,n):
    return [[x / n for x in obj] for obj in L1]

def square(L1):
    return [[x ** 2 for x in obj] for obj in L1]

def squareroot(L1):
    return [[x ** 0.5 for x in obj] for obj in L1]

def openup(L): # accept a list with tuple, return with a list without tuple
    temp = []
    for sys in L:
        temp = temp + list(sys)
    return np.asarray(temp)

def magnitude(flux):  # return the magnitude correspond to the flux, input number only
    M = -2.5*(np.log10(flux)) + m_zp
    return M

def MAGNITUDE(L):  # whole object
    return [[magnitude(x) for x in obj] for obj in L]


lum = multiply(adu, gain)  #change the total image flux to e-s^-1 unit

print(PLUS(test1,test2))
print(plus(test1,1))
print(MINUS(test1,test2))
print(minus(test1,1))
print(MULTIPLY(test1,test2))
print(multiply(test1,1))
print(DIVIDE(test1,test2))
print(divide(test1,2))
print(magnitude(40))
print(MAGNITUDE(lum))

[[9, 3], [14, 137, 116], [1.1, 5, 4.1]]
[[2, 3], [6, 56, 8], [2, 5, 1.1]]
[[-7, 1], [-4, -27, -102], [0.9, 3, -3.9]]
[[0, 1], [4, 54, 6], [0, 3, -0.9]]
[[8, 2], [45, 4510, 763], [0.1, 4, 0.4]]
[[1, 2], [5, 55, 7], [1, 4, 0.1]]
[[0.125, 2.0], [0.5555555555555556, 0.6707317073170732, 0.06422018348623854], [10.0, 4.0, 0.025]]
[[0.5, 1.0], [2.5, 27.5, 3.5], [0.5, 2.0, 0.05]]
20.6897500217
[[19.693942260339796, 20.026946009524892, 20.455993346606451], [21.483317076477526, 21.241497146765518, 21.921173227586884], [23.822004781502379, 23.015068348642405], [23.700049978319907, 20.999480280585498, 22.343262197882169], [24.160323072992107, 24.532110068878605], [23.173271052800505, 23.117167598236399, 24.706630241271053], [21.146191781602617, 21.78475445316759, 22.71274063871158], [20.996167096566062, 21.003260775350672], [22.00328687283065, 23.848196694779844, 23.8446803215598]]


In [59]:
A = [1,2,3]
print(3*A)

[1, 2, 3, 1, 2, 3, 1, 2, 3]
